# Programming assignment 1

You should submit your python code as a solution. Use the PuLP library https://pypi.org/project/PuLP/ to solve the following problems. Documentation to PuLP can be found here: https://coin-or.github.io/pulp/main/index.html

A few hints To surpress the messages from the solver, you can use `GLPK(msg = 0)` (if you use GLPK) or `PULP_CBC_CMD(msg=False)` (if you use CBC) or a similar option for other solvers as a solver parameter of `solve()` method.

You can use function `value()` to evaluate any expression involving the variables in the optimal solution. If you need many variables, it might be good to introduce them using an array of their names using `dicts()` method and use the returned dictionary to access them.



### Part 1

Solve the following program:

Objective:
`min 122x + 143y`

Subject to:
1. `x ≥ -10`
2. `y ≤  10`
3. `3x +  2y ≤  10`
4. `12x + 14y ≥ -12.5`
5. `2x +  3y ≥   3`
6. `5x -  6y ≥ -100`

Required output example:
- Optimal solution: `x = 0.1 y = -2.3`
- Objective value: `100.1214`
- Tight constraints: 1, 2, 4
- Unique optimal solution: `NO`

So, your program should find the optimal solution, determine its objective value, identify the tight constraints and decide whether the optimal solution is unique.
Note that, in the problem, there are only two directions perpendicular to the objective.


In [ ]:
from pulp import *

# Define the problem
exer1 = pulp.LpProblem("exer1", LpMinimize)

# Define the decision variables
x = LpVariable("x")
y = LpVariable("y")

# Define the objective function
exer1 += 122 * x + 143 * y

# Define the constraints
exer1 += x >= -10
exer1 += y <= 10
exer1 += 3 * x + 2 * y <= 10
exer1 += 12 * x + 14 * y >= -12.5
exer1 += 2 * x + 3 * y >= 3
exer1 += 5 * x - 6 * y >= -100

# Solve the problem
exer1.solve()

# Save the constraints
constraints = []

for constraint in exer1.constraints.values():
    constraints.append(constraint)

# Save the tight constraints
tight_constraints = []
    
for i in range(6):
    if constraints[i].slack == 0:
        tight_constraints.append(i+1)

# Print the results
print("Objective value:", value(exer1.objective))
for variable in exer1.variables():
    print(variable.name, "=", variable.varValue)
print("Tight constraints are:", tight_constraints)

### Part 2

Find an optimal mixed strategy of the following game:

Both players choose independently a single integer from 1 to 6. Then, the numbers are compared:

- If they are equal, there is a draw 
- If they differ by 1, the player who played the smaller number gets 2EUR from the other player 
- If they differ by ≥2, the player who played the larger number gets 1EUR from the other player 

Note that the game is symmetric and the same strategy is optimal for both players.

**Required output example:**

- **x1:** 0.2 
- **x2:** 0.1 
- **x3:** 0.2 
- **x4:** 0.1 
- **x5:** 0.2 
- **x6:** 0.2


In [ ]:
from pulp import *

# Define the model
exer2 = LpProblem("exer2", LpMaximize)

# Define the variables
x1 = LpVariable("x1", lowBound=0, upBound=1, cat='Continuous')
x2 = LpVariable("x2", lowBound=0, upBound=1, cat='Continuous')
x3 = LpVariable("x3", lowBound=0, upBound=1, cat='Continuous')
x4 = LpVariable("x4", lowBound=0, upBound=1, cat='Continuous')
x5 = LpVariable("x5", lowBound=0, upBound=1, cat='Continuous')
x6 = LpVariable("x6", lowBound=0, upBound=1, cat='Continuous')
x0 = LpVariable("x0", cat='Continuous')

# Define the objective function
exer2 += x0

# Define the constraints
exer2 += x0 <= -2*x1+x3+x4+x5+x6
exer2 += x0 <= 2*x1-2*x3+x4+x5+x6
exer2 += x0 <= -x1+2*x2-2*x4+x5+x6
exer2 += x0 <= -x1-x2+2*x3-2*x5+x6
exer2 += x0 <= -x1-x2-x3+2*x4-2*x6
exer2 += x0 <= -x1-x2-x3-x4+2*x5
exer2 += x1+x2+x3+x4+x5+x6 >= 1
exer2 += x1+x2+x3+x4+x5+x6 <= 1
exer2 += x1 >= 0
exer2 += x2 >= 0
exer2 += x3 >= 0
exer2 += x4 >= 0
exer2 += x5 >= 0
exer2 += x6 >= 0
# Solve the model
exer2.solve()

# Print the results
print("Optimal obejctive:")
print(f"x0 = {x0.varValue}")
print("Optimal mixed strategy:")
print(f"x1 = {x1.varValue}")
print(f"x2 = {x2.varValue}")
print(f"x3 = {x3.varValue}")
print(f"x4 = {x4.varValue}")
print(f"x5 = {x5.varValue}")
print(f"x6 = {x6.varValue}")

### Part 3

On an Imaginary Island's Contract Inspection

On some imaginary island, there are 69 companies and bilateral contracts between them. The monarch of the island wishes to inspect the validity of each of these contracts during one grand event. For each contract, the monarch demands two representatives to stand for the contract relationship (they can either be from the same party of the contract or one from each party). By the default approach, each company would send one representative, hence requiring a total of 69 representatives. However, the companies aim to determine a solution that reduces the overall number of representatives present.

The input file, `hw1-03.txt`, provides details about these contracts. Each line denotes a separate contract and includes identifiers (ranging from 1-69) of the two involved parties, separated by a space.

Example Output:

- **representatives from company 1:** 1.0
- **representatives from company 2:** 2.0
- **representatives from company 3:** 1.0
...
- **representatives from company 69:** 1.0

**Total number of representatives involved:** 58

> **Important:** Sending a fractional representative is not permissible. However, the LP relaxation's solution suffices, as it already yields an integral outcome. The justification behind this will be elaborated upon in subsequent classes.


In [ ]:
import pulp

# Define the model
problem = pulp.LpProblem("Contracts", pulp.LpMinimize)

x = []
contr = []

# Define the variables

for i in range(69):
    x.append(i)

for i in range(69):
    x[i] = pulp.LpVariable("x_{}".format(i+1), lowBound=0, cat='Discrete')

file = "hw1-03.txt"
with open(file, "r") as file:
    lines = file.readlines()

lines = [line.strip() for line in lines]

for line in lines:
    contr.append(line)

# Define the objective function
problem += sum(x)

# Define the constraints

for i in range(len(contr)):
    a, b = contr[i].split()
    a = int(a)
    b = int(b)
    problem += x[a-1] + x[b-1] >= 2

# Solve the problem
problem.solve()

# Print the results
print("Objective value:", pulp.value(problem.objective))

for variable in problem.variables():
    print(variable.name, "=", variable.varValue)